# <center>Day 4: matching models with nontransferable utility</center>
### <center>Alfred Galichon (NYU)</center>
## <center>'math+econ+code' masterclass on equilibrium transport and matching models in economics</center>
<center>© 2020 by Alfred Galichon.  Support from  NSF DMS-1716489 and ERC CoG-866274 EQUIPRICE grants is acknowledged.</center>

#### <center>with Python code</center>

**If you reuse code from this masterclass, please cite as:**<br>
Alfred Galichon, 'math+econ+code' masterclass on equilibrium transport and matching models in economics, June 2020. https://github.com/math-econ-code/mec_equil


# References

## Textbooks

Alvin Roth and Marilda Sotomayor (1990). *Two-sided matching*. Econometric Society Monographs, Cambridge University Press.

## Papers

David Gale and Lloyd Shapley (1962). "College Admissions and the Stability of Marriage." *American Mathematical Monthly* 69 (1), pp. 9–14.

Hiroyuki Adachi (2000). "On a characterization of stable matchings." *Economics Letters* 68 pp. 43–49.

Alfred Galichon and Yu-Wei Hsieh (2019)."Aggregate stable matching with money burning." SSRN id=2887732. 


# Matching without transfers

## Generating the data

We will generate the same worker/firm data as yesterday.

In [1]:
import numpy as np
from scipy.spatial.distance import cdist

np.random.seed(777)
d = 8
nbx = 5 #1200
nby = 3 #1000

rg = .8
rs = .6
r  = .7

n_x = np.ones(nbx)
m_y = np.ones(nby)
ξ_x_k = np.random.rand(nbx,d)
ζ_y_k = np.random.rand(nby,d)

α_x_y = np.zeros((nbx,nby))
γ_x_y = np.zeros((nbx,nby))
for x in range(nbx):
    for y in range(nby):
        α_x_y[x,y] = - np.linalg.norm(ξ_x_k[x,6:7]-ζ_y_k[y,6:7])
        γ_x_y[x,y] = 50*(np.sum( (ξ_x_k[x,0:3]*ζ_y_k[y,0:3])**rg )**(r/rg)+(  np.sum(  ξ_x_k[x,3:6]*ζ_y_k[y,3:6] )**rs )**(r/rs))**r
        

class GaleShapley_model:
    def __init__(self,α_x_y,γ_x_y,n_x = np.array([]), m_y = np.array([])):
        self.α_x_y = α_x_y
        self.γ_x_y = γ_x_y
        nbx = α_x_y.shape[0]
        nby = α_x_y.shape[1]        
        if n_x.size == 0:
            n_x = np.ones(nbx)
        if m_y.size ==  0:
            m_y = np.ones(nby) 
        self.n_x = n_x
        self.m_y = m_y
        self.largex = nby
        self.largey = nbx
        self.smallx = -1
        self.smally = -1
        self.nbx = nbx
        self.nby = nby
        self.αo_x_y = np.zeros((nbx,nby), dtype = 'int64') 
        self.γo_x_y = np.zeros((nbx,nby), dtype = 'int64') 
        self.prefslistα_x_y = np.zeros((nbx,nby), dtype = 'int64') 
        self.prefslistγ_x_y = np.zeros((nbx,nby), dtype = 'int64') 

        for x in range(nbx):
            thelistx = (- α_x_y )[x,:].argsort()
            self.αo_x_y[x, thelistx] =  nby - 1 - np.arange(nby)
            self.prefslistα_x_y[x,:] = thelistx
        for y in range(nby):
            thelisty = ( - γ_x_y)[:,y].argsort()
            self.γo_x_y[thelisty, y] = nbx - 1 - np.arange(nbx)
            self.prefslistγ_x_y[:,y] = thelisty
        self.comp_nbsteps = -1
        self.comp_time = -1
        self.eq_μ_x_y = np.array([])
        
    def print_prefs(self,xs=[],ys=[]):
        if xs == [] and ys==[] :
            xs = range(self.nbx)
            ys = range(self.nby)
        if xs != [] :
            for x in xs :
                xsprefs = 'x' + str(x) +': y' + str(np.where(self.αo_x_y[x,:]== (nby - 1))[0][0] )
                for y in range(nby-1):
                    xsprefs = xsprefs + ' > y' + str( np.where(self.αo_x_y[x,:]== (nby - y - 2))[0][0] )
                print(xsprefs)
        
        if xs != [] and ys != [] :
            print('===')
        
        if ys != [] :
            for y in ys :
                ysprefs = 'y' + str(y) +': x' + str(np.where(self.γo_x_y[:,y]== (nbx - 1))[0][0] )
                for x in range(nbx-1):
                     ysprefs = ysprefs + ' > x' + str( np.where(self.γo_x_y[:,y]== (nbx - x - 2))[0][0] )
                print(ysprefs)
    

##################
mkt = GaleShapley_model(α_x_y,γ_x_y)
mkt.print_prefs()
        

x0: y0 > y1 > y2
x1: y0 > y1 > y2
x2: y2 > y1 > y0
x3: y2 > y1 > y0
x4: y0 > y1 > y2
===
y0: x3 > x4 > x0 > x2 > x1
y1: x3 > x4 > x2 > x0 > x1
y2: x3 > x4 > x0 > x1 > x2


## Stable matchings

The following function detects stable matchings.

In [2]:
def is_stable(self, μ_x_y = np.array([]),output=0 ):
    if μ_x_y.size == 0 :
        if output > 0 :
            print('No μ_x_y passed, taking the attached eq_μ_x_y instead.' )
        μ_x_y = self.eq_μ_x_y
    uo_x = np.zeros(self.nbx)
    vo_y = np.zeros(self.nby)        
    for x in range(self.nbx):
        if (np.min(μ_x_y[x,:]) < 0) or (np.sum(μ_x_y[x,:])>1) :
            print('μ is not conform.')
            return(False)
        yxs = np.where(μ_x_y[x,:]==1)[0]
        if yxs.size == 0 :
            uo_x[x] = self.smallx
        elif yxs.size == 1 :
            uo_x[x] =  self.αo_x_y[x,yxs[0]]
        else :
            print('μ is not conform.')
            return(False)
    # end for x
    
    for y in range(self.nby):
        if np.min(μ_x_y[:,y])< 0 or np.sum(μ_x_y[:,y])>1:
            print('μ is not conform.')
            return(False)
        xys = np.where(μ_x_y[:,y]==1)[0]
        if xys.size == 0 :
            vo_y[y] = self.smally
        elif xys.size == 1 :
            vo_y[y] =  self.γo_x_y[xys[0],y]
        else :
            print('μ is not conform.')
            return(False)
    # end for y
    
    for x in range(x):
        for y in range(y):
            if (self.αo_x_y[x,y] > uo_x[x]) and (self.γo_x_y[x,y] > vo_y[y]):
                if output > 0 :
                    print('The matching is not stable, there is at least one blocking pair: x'+str(x),'y'+str(y)+'.')
                return(False)
    # end for x and y
    if output > 0 :
        print('The matching is stable.')
    return (True)


GaleShapley_model.is_stable = is_stable


## Deferred acceptance: Gale and Shapley's algorithm

Reference:
* David Gale and Lloyd Shapley (1962). "College Admissions and the Stability of Marriage." *American Mathematical Monthly* 69 (1), pp. 9–14.


In [3]:
def solveGaleShapley(self,output=0):
    μA_x_y = np.ones((self.nbx, self.nby)) # initially all offers are non rejected
    while True :
        μP_x_y = np.zeros((self.nbx, self.nby)) 
        # Each x makes an offer to their favorite y: 
        μP_x_y[range(self.nbx), np.argmax(self.αo_x_y + np.where( μA_x_y , 0 , -self.largex ),axis =1 )] = 1         
        μP_x_y = np.minimum(μP_x_y , μA_x_y) # if no offer is available to x, then x makes no offer
        
        μE_x_y = np.zeros((self.nbx, self.nby))
        # Each y retains their favorite offer among those made:
        μE_x_y[ np.argmax(self.γo_x_y + np.where( μP_x_y , 0 , -self.largey ),axis =0 ),range(self.nby)] = 1
        μE_x_y = np.minimum(μE_x_y , μP_x_y) # if no offer is made to y, then y retains no offer
        rej_x_y = μP_x_y - μE_x_y # compute rejected offers
        if np.max(np.abs(rej_x_y)) == 0: 
            break # if all offers have been accepted, then algorithm stops
        μA_x_y = μA_x_y - rej_x_y  # offers from x that have been rejected are no longer available to x
        if output >= 2:
            print("μ_P=, μ_E=" )
            print(μP_x_y)
            print(μE_x_y)
        self.comp_nbsteps +=1
        self.eq_μ_x_y = μE_x_y
    return (0)

GaleShapley_model.solveGaleShapley = solveGaleShapley 

## Deferred acceptance revisited: Adachi's algorithm

In [4]:
def solveAdachi(self,output=0):
    uo_x = self.largex * np.ones(self.nbx, dtype = 'int64') # x's utilities are highest
    vo_y = self.smally * np.ones(self.nby, dtype = 'int64') # y's utilities are lowest
    while True :
        # each x proposes to favorite y among those willing to consider them:
        uonew_x = np.max(self.αo_x_y + np.where( (self.γo_x_y >= vo_y) , 0 , -self.largex ),axis =1 )  
        if (uonew_x == uo_x).all() :
            break
        uo_x = uonew_x      
        # each y proposes to favorite x among those willing to consider them:
        vo_y = np.max(self.γo_x_y + np.where( (self.αo_x_y.transpose() >= uo_x).transpose() , 0, -self.largey) ,axis = 0)
        if output >= 2:
            μP_x_y = np.where( (self.αo_x_y.transpose() == uo_x).transpose() , 1 , 0 )
            μE_x_y = np.where( self.γo_x_y == vo_y , 1 , 0 )
            print("μ_P=, μ_E=" )
            print(μP_x_y)
            print(μE_x_y)
        self.comp_nbsteps += 1
    self.eq_μ_x_y = np.where( self.γo_x_y == vo_y , 1 , 0 )
    return(0)

GaleShapley_model.solveAdachi = solveAdachi 

## Aggregate stable matching: DARUM

Reference:
* Alfred Galichon and Yu-Wei Hsieh (2019)."Aggregate stable matching with money burning." SSRN id=2887732.

In [5]:
def proposeNoHet(self,axis,μbar_x_y):
    if axis == 0 : # if proposing side = x
        n_x = self.n_x
        nbx = self.nbx
        nby = self.nby
        prefs_x_y = self.prefslistα_x_y
    else:
        n_x = self.m_y
        nbx = self.nby
        nby = self.nbx
        prefs_x_y = self.prefslistγ_x_y.transpose()    
    μ_x_y = np.zeros((nbx,nby))
    for x in range(nbx):
        nxres = n_x[x]
        for yind in prefs_x_y[x,]:
            if μbar_x_y[x,yind] > 0:
                μ_x_y[x,yind] = min(nxres , μbar_x_y[x,yind])
                nxres -= μ_x_y[x,yind]
            if nxres == 0:
                break
    return(μ_x_y)

GaleShapley_model.proposeNoHet = proposeNoHet

def propose(self,axis,μbar_x_y,heterogeneity = 'none'):
    if heterogeneity == 'none':
        μ_x_y = self.proposeNoHet(axis,μbar_x_y)
    else:
        raise Exception("Heterogeneity " + heterogeneity + " is not supported.")
    return (μ_x_y)

GaleShapley_model.propose = propose

def solveDARUM(self,output=1, tol = 1e-5):
    μA_x_y = np.zeros((self.nbx, self.nby)) # initially all offers are non rejected
    for x in range(self.nbx):
        for y in range(self.nby):
            μA_x_y[x,y] = max(self.n_x[x],self.m_y[y])
    while True :
        μP_x_y = self.propose(0 , μA_x_y,'none') # the x's pick their preferred offers among those not rejected
        μE_x_y = self.propose(1, μP_x_y.transpose(),'none').transpose() # the y's pick their preferred offers among those made
        rej_x_y = μP_x_y - μE_x_y # compute rejected offers
        if np.max(np.abs(rej_x_y)) < tol: 
            break # if all offers have been accepted (within tolerange), then algorithm stops
        μA_x_y = μA_x_y - rej_x_y  # offers from x that have been rejected are no longer available to x
        if output >= 2:
            print("μ_P=, μ_E=" )
            print(μP_x_y)
            print(μE_x_y)
        self.comp_nbsteps +=1
        self.eq_μ_x_y = μE_x_y
    return (0)

GaleShapley_model.solveDARUM = solveDARUM 

## Comparing Adachi, Gale and Shapley, and DARUM

The following compares the Gale-Shapley and Adachi algorithms.

In [6]:
from time import time

def solveDeferredAcceptance(self, algorithm = 'Adachi', output=0):
    start_time = time()
    self.comp_nbsteps = 0
    if algorithm == 'GS' :
        self.solveGaleShapley(output)
    elif algorithm == 'Adachi' :
        self.solveAdachi(output)
    elif algorithm == 'DARUM' :
        self.solveDARUM(output)
    else:
        raise Exception("Algorithm " + algorithm + " is not implemented.")
    self.comp_time =  time() - start_time
    if output >= 1:
        print("Converged in ",self.comp_nbsteps," steps and ", self.comp_time, " seconds.")
    if output == 1:
        print("μ_x_y=",self.eq_μ_x_y) 
    return (0)

    
GaleShapley_model.solveDeferredAcceptance = solveDeferredAcceptance
##################
#res=mkt.solveDeferredAcceptance(algorithm = 'GS', output=2)

In [15]:
nbx=100
nby=80
seed=80809
print("Times: Gale-Shapley, Adachi, Darum")
for incr in range(5):
    np.random.seed(seed+incr)
    α_x_y = np.random.rand(nbx,nby)
    γ_x_y = np.random.rand(nbx,nby)
    tstmkt = GaleShapley_model(α_x_y,γ_x_y)
    res=tstmkt.solveDeferredAcceptance(algorithm = 'GS')
    i1 = tstmkt.comp_nbsteps
    t1 = tstmkt.comp_time
    mu1 = tstmkt.eq_μ_x_y
    res=tstmkt.solveDeferredAcceptance(algorithm = 'Adachi')
    i2 = tstmkt.comp_nbsteps
    t2 = tstmkt.comp_time
    mu2 = tstmkt.eq_μ_x_y
    res=tstmkt.solveDeferredAcceptance(algorithm = 'DARUM')
    i3 = tstmkt.comp_nbsteps
    t3 = tstmkt.comp_time
    mu3 = tstmkt.eq_μ_x_y
    if not tstmkt.is_stable() :
        raise Exception('Matching is not stable')
    if  np.max(np.abs(mu1 - mu2))+ np.max(np.abs(mu1 - mu3)) != 0:
        raise Exception(seed,': results differ.')
    print("incr ", incr)
    print("nbsteps", i1,i2,i3)
    print("time", t1,t2,t3)
print('Done')


Times: Gale-Shapley, Adachi, Darum
incr  0
nbsteps 467 28 467
time 0.11818981170654297 0.0 1.9655382633209229
incr  1
nbsteps 313 22 313
time 0.07062244415283203 0.0 0.9697239398956299
incr  2
nbsteps 236 18 236
time 0.04812169075012207 0.0 0.6645801067352295
incr  3
nbsteps 306 22 306
time 0.07499957084655762 0.0030002593994140625 0.9528963565826416
incr  4
nbsteps 294 22 294
time 0.07939839363098145 0.005017280578613281 0.934110164642334
Done
